In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn.functional as F
import torch.optim as optim

In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import gym

In [24]:
from gym import Env
from gym.spaces import Discrete, Box,Tuple,MultiDiscrete,MultiBinary
import random
import os

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
import numpy as np
import torch

path_to_data = "/gdrive/MyDrive/MIE1666project/rl-ccmvo-main/rl-ccmvo-main/Data/synthetic1000"
date = "2016-02-07"
mu = np.loadtxt('{}/{}/mu'.format(path_to_data, date))
Q = np.loadtxt('{}/{}/Q'.format(path_to_data, date))
# Q/=torch.linalg.norm(torch.tensor(Q))
# mu/=torch.linalg.norm(torch.tensor(mu))

# Normalization of Q and mu

# Three important function for the RL env set-up, **MVO(mu,Q,gamma), generate_Q(Q,index) and derivative(mu,Q,gamma,w)**


*   MVO(mu,Q,gamma) returns the weight vector and the negative MVO (return-risk) value.
*   generate_Q(Q,index) is used in each time step for the agent to generate a **Q** convariance for allocated k assets.
*   derivative(mu,Q,gamma,w) is used to give the derivative MVO which is used to provide the reward flag for +1 if derivative >0 or else reward=-1.




In [6]:
import cvxpy as cp
def MVO(mu,Q,gamma):
    # Convert mu to NumPy array if it's not already
  if not isinstance(mu, np.ndarray):
      mu = np.array(mu)

  # Convert Q to NumPy array if it's not already
  if not isinstance(Q, np.ndarray):
      Q = np.array(Q)
  if mu.size == 0 or Q.size == 0:
    return 0,0
  else:
    n = len(mu)

    # Decision Variables
    w = cp.Variable(n)

    constraints = [
        cp.sum(w) == 1, # Sum to 1
        w>=0 # Disallow Short Sales
    ]

    # Objective Function
    risk = cp.quad_form(w, Q)
    targetRet=gamma*mu.T@w
    prob = cp.Problem(cp.Maximize(targetRet - risk), constraints=constraints)
    prob.solve()
    return w.value,targetRet.value-risk.value

In [7]:
# function to select Qs from the Q array of 1000/500 el
# def generate_Q(Q,index):
#   # Use np.meshgrid to generate all pairs of indices
#   all_pairs = np.array(np.meshgrid(index,index)).T.reshape(-1, 2)

#   # Use advanced indexing to select the elements
#   selected_elements = Q[all_pairs[:, 0], all_pairs[:, 1]]
#   size=int(np.sqrt(selected_elements.size))
#   return selected_elements.reshape(size,size)
def generate_Q(Q, index):
    # Convert index to a PyTorch tensor
    index_tensor = torch.tensor(index)

    # Use torch.meshgrid to generate all pairs of indices
    all_pairs = torch.meshgrid(index_tensor, index_tensor)
    all_pairs = torch.stack(all_pairs, dim=-1).reshape(-1, 2)

    # Use advanced indexing to select the elements
    selected_elements = Q[all_pairs[:, 0], all_pairs[:, 1]]

    # Calculate the size based on the length of the index array
    size = len(index)

    return selected_elements.reshape(size, size)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
# derivative function always return the gradient of MVO wrt the last x added to the list
def derivative(mu,Q,gamma,w):
  # if mu.size <= 1 or Q.size <= 1:
  #   return -gamma*mu[0]
  # else:
  risk=0
  for i in range(len(mu)):
      risk+=Q[i][-1]*w[i]
  Ret=mu[-1]*gamma
      # Ret+=mu[-1]*gamma
  return Ret-risk


In [9]:
# derivative function always return the gradient of MVO wrt the last x added to the list
def Avgderivative(mu,Q,gamma,w):
  # if mu.size <= 1 or Q.size <= 1:
  #   return -gamma*mu[0]
  # else:
  result=[]
  for i in range(len(Q)):
    risk=0
    for j in range(len(mu)):
        risk+=Q[i][j]*w[i]
        Ret=mu[j]*gamma
        # print(Q[i][j],mu[j])
    result.append(Ret-risk)
        # Ret+=mu[-1]*gamma
  return np.mean(result)

In [10]:
Q2=generate_Q(Q,[1,2,3])

mu2=mu[1:4]
Q2,mu2

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


(array([[29.2127241 , 20.03018647, 19.97611704],
        [20.03018647, 30.5748112 , 20.28661393],
        [19.97611704, 20.28661393, 29.90946638]]),
 array([0.01406372, 0.05210043, 0.0527893 ]))

In [11]:
w2,_=MVO(mu2,Q2,1)

In [12]:
Avgderivative(mu2,Q2,1,w2)

-23.296618217000574

In [13]:
class TrainEnv(Env):
## initialize parameter:##
    def __init__(self, Q):

        self.index = [] #index of the latest asset added to allocation
        self.cur_mu = [] #cur_mu and cur_Q at time step k
        self.cur_Q = []
        self.action_space = Discrete(len(mu)) # action_space and observation _space are defined, action space should be 1000, e.g 0 means the first asset is selected
        self.observation_space = Box(low=-np.inf, high=np.inf)
        # state is a numpy array float
        self.state =np.array([0.]) # Initialize state which should be state[0]= MVO if updated
        self.asset_length = 50 # asset length to end
        self.used_actions = []  # Keep track of used actions in the current iteration

    def step(self, action, done):
        if done == True:
          self.used_actions=set()
          self.index = [] #index of the latest asset added to allocation
          self.cur_mu = [] #cur_mu and cur_Q at time step k
          self.cur_Q = []
          # self.mvo_old=0
        # action is the index of the asset
        # action = self.sample_unique_action()
        self.used_actions.add(action)
        gamma = 1
        self.index.append(action)
        self.cur_mu.append(mu[action])
        self.cur_Q=generate_Q(Q,self.index) # Assuming generate_Q is defined
        # print("cur mu:",len(self.cur_mu),"cur_Q",self.cur_Q.size())
        w, self.state[0] = MVO(self.cur_mu, self.cur_Q, gamma)  # Assuming MVO is defined
        self.asset_length -= 1
        self.transition= Avgderivative(self.cur_mu,self.cur_Q, gamma, w)  # Assuming derivative is defined

        # Calculate reward
        # reward=self.transition.item()
        if self.transition.item()<=0:
          reward = -1
        elif self.index:
          reward=-10
        else:
          reward=1
        # mvo_old=self.mvo_new
        done = self.asset_length <= 0

        # Set placeholder for info
        info = {}

        # Return step information
        return self.state, reward, done, info

    # Make sure each action are unique
    def sample_unique_action(self):
      # Sample a new action until it is unique in the current iteration
      new_action = self.action_space.sample()
      while new_action in self.used_actions:
          new_action = self.action_space.sample()
      return new_action
    # def render(self):
    #     # Implement viz
    #     pass

    def reset(self):
        # Reset
        self.state = np.array([0.])
        self.used_actions.clear()
        self.asset_length = 50
        return self.state


In [14]:
n=len(mu)
n_actions=n
K=2
episode=100

In [15]:
def find_MVO(mu,Q,w,gamma):
    # Objective Function
    # Q=Q.numpy()
    # mu=mu.numpy()
    risk = w@Q@w
    targetRet=gamma*mu.T@w
    return targetRet-risk

# DQN model (not Stable)

We will need two Neural network in DQN, one is for evaluation and one is for forward pass (Target).

In [17]:
# Some new stuff
class Net(nn.Module):
    def __init__(self, input_size, num_classes):
        super(Net, self).__init__()

        # Shared layers
        self.shared_layers = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )

        # Discrete output branch
        self.discrete_output = nn.Linear(64, num_classes)

        # Continuous output branch
        self.continuous_output = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.shared_layers(x)

        # Discrete output
        discrete_result = F.softmax(self.discrete_output(x), dim=1)

        # Continuous output
        continuous_result = F.sigmoid(self.continuous_output(x))

        return discrete_result, continuous_result


# DQN approach (not stable)

In [19]:
class DQN(object):
    def __init__(self):
        self.eval_net, self.target_net = Net(), Net()
        self.learn_step_counter = 0                                     # for target updating
        self.memory_counter = 0                                         # for storing memory
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 2))     # initialize memory
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)
        self.loss_func = nn.MSELoss()
        self.used_actions = set()  # Keep track of used actions in the current iteration


    def choose_action(self,x,done):
        x = torch.unsqueeze(torch.FloatTensor(x), 0)
        # input only one sample
        if done == True:
          # print(self.used_actions)
          self.used_actions=set()
        if np.random.uniform() < EPSILON:   # greedy
            # print("it's greedy")
            actions_value = self.eval_net.forward(x)
            for i in self.used_actions:
                actions_value[0][i] = -1000 #assign a large negative value for selected action such that it wont select it again
            action = torch.max(actions_value, 1)[1].data.numpy()
            action = action[0]
            # if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)  # return the argmax index

        else:   # random
            # print("it's random")
            available_actions = list(set(range(N_ACTIONS))- self.used_actions)
            action = np.random.choice(available_actions)
            action = action
            # if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)
        self.used_actions.add(action)


        # print("used action:",self.used_actions)
        # print("Used action:",self.used_actions,"action for this iter:",action, "ENV_A_SHAPE:",ENV_A_SHAPE)
        return action

    def store_transition(self, s, a, r, s_):
        transition = np.hstack((s, [a, r], s_))
        # replace the old memory with new memory
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition
        self.memory_counter += 1
    def learn(self):
        # target parameter update
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.learn_step_counter += 1

        # sample batch transitions
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        b_memory = self.memory[sample_index, :]
        # print(b_memory)
        b_s = torch.FloatTensor(b_memory[:, :N_STATES])
        b_a = torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int))

        b_r = torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2])

        b_s_ = torch.FloatTensor(b_memory[:, -N_STATES:])

        # q_eval w.r.t the action in experience
        q_eval = self.eval_net(b_s).gather(1, b_a)  # shape (batch, 1)
        q_next = self.target_net(b_s_).detach()     # detach from graph, don't backpropagate
        q_target = b_r + GAMMA * q_next.max(1)[0].view(BATCH_SIZE, 1)   # shape (batch, 1)
        loss = self.loss_func(q_eval, q_target)
        self.optimizer.zero_grad()

        loss.backward()
        self.optimizer.step()


In [20]:
# For trainenv2 the state is initialized with the MVO--> dim (1,)
#and the action is the selection of the 50 stock (50 out of 1000 binary array)
# reward is the exp of last state over the current state

class TrainEnv2(Env):
## initialize parameter:##
    def __init__(self,n,K,episode):
      self.index=np.random.choice(n,K,replace=False)
      self.cur_mu = mu[self.index] #cur_mu and cur_Q at time step k
      self.cur_Q = generate_Q(Q,self.index)

      # nVec=[n]*K
      self.action_space = MultiBinary(n)
      self.observation_space = Box(low=-np.inf, high=np.inf)

      self.weight,self.state =MVO(self.cur_mu,self.cur_Q,1) # Initialize state which should be state[0]= MVO if updated
      self.episode_length=episode
    # Given the agent's action (an array of action), returns
    # Make sure each action are unique
    def unique_action(self,index):
      return len(np.unique(index))==len(index)

    def step(self, action):
      self.old_state=self.state
      self.index=np.nonzero(action)
      if len(self.index[0])!=K:
        reward = -1000
      else:
        self.cur_mu = mu[self.index[0]]  # cur_mu and cur_Q at time step k
        self.cur_Q = generate_Q(Q, self.index[0])

        self.weight,self.state = MVO(self.cur_mu, self.cur_Q, 1)
        self.episode_length-=1


        # Calculate reward
        self.transition=self.state-self.old_state
        if self.transition>0:
          reward=1
        elif self.transition==0:
          reward=0.5
        else:
          reward=-1

        # if not self.unique_action(self.index):
        #     reward = -1000

      # Check if episode is done
      done = self.episode_length <= 0
      if done:
          self.reset()  # Reset if done



      self.episode_length -= 1

      # Set placeholder for info
      info = {}

      # Return step information
      return self.state, reward, done, info
    def reset(self):
      # Reset
      self.index=np.random.choice(n,K,replace=False)
      self.cur_mu = mu[self.index] #cur_mu and cur_Q at time step k
      self.cur_Q = generate_Q(Q,self.index)
      self.weight,self.state =MVO(self.cur_mu,self.cur_Q,1) # Initialize state which should be state[0]= MVO if updated
      self.episode_length=episode
      return self.state





```

#Dueling DQN (Dueling Q learning)**

In [25]:
date_ls=[]
for file in os.listdir(path_to_data):
    date_ls.append(file)
# mu = np.loadtxt('{}/{}/mu'.format(path_to_data, date))
# Q = np.loadtxt('{}/{}/Q'.format(path_to_data, date))
date_ls[0]

'2016-02-14'

In [26]:
import os
class replay_buffer():
  def __init__(self,max_size,input_shape):
    self.mem_size=max_size
    self.memory_counter=0
    self.state_memory=np.zeros((self.mem_size,*input_shape),dtype=np.float32)
    self.newstate_memory=np.zeros((self.mem_size,*input_shape),dtype=np.float32)
    self.action_memory=np.zeros((self.mem_size,n),dtype=np.int64)
    self.reward_memory=np.zeros(self.mem_size,dtype=np.float32)
    self.terminal_memory=np.zeros(self.mem_size,dtype=np.float32)
  def store_transition(self,state,action,reward,state_,done):
    index=self.memory_counter%self.mem_size
    self.state_memory[index]=state
    self.newstate_memory[index]=state_
    self.reward_memory[index]=reward
    self.action_memory[index]=action
    self.terminal_memory[index]=done
    self.memory_counter+=1
  def sample_buffer(self,batch_size):
    max_mem=min(self.memory_counter,self.mem_size)
    batch=np.random.choice(max_mem,batch_size,replace=False)
    states=self.state_memory[batch]
    actions=self.action_memory[batch]
    rewards=self.reward_memory[batch]
    states_=self.newstate_memory[batch]
    terminal=self.terminal_memory[batch]
    return states,actions,rewards,states_,terminal

class dueling_double_DQN(nn.Module):
  def __init__(self,lr,n_actions,name,input_dims,chkpt_dir):
    super(dueling_double_DQN,self).__init__()
    self.chkpt_dir=chkpt_dir
    self.checkpoint_file=os.path.join(self.chkpt_dir,name)

    self.fc1=nn.Linear(*input_dims,512)
    self.V=nn.Linear(512,1)
    self.A=nn.Linear(512,n_actions)

    self.optimizer=optim.Adam(self.parameters(),lr=lr)
    self.loss=nn.MSELoss()
    self.device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  def forward(self,state):
    flat1=F.relu(self.fc1(state))
    V=self.V(flat1)
    A=self.A(flat1)

    return V,A
  def save_check_point(self):
    print("---Saving Checkpoint")
    torch.save(self.state_dict(),self.checkpoint_file)
  def load_check_point(self):
    print('---Loading Checkpoint')
    self.load_state_dict(torch.load(self.checkpoint_file))

class Agent():
  def __init__(self,gamma,epsilon,lr,n_actions,input_dims,
              mem_size,batch_size,eps_min=0.01,eps_dec=5e-7,
              replace=100,chkpt_dir='/content'):
    self.gamma=gamma
    self.epsilon=epsilon
    self.lr=lr
    self.n_actions=n_actions
    self.input_dims=input_dims
    self.batch_size=batch_size
    self.eps_min=eps_min
    self.eps_dec=eps_dec
    self.replace_target_cnt=replace
    self.chkpt_dir=chkpt_dir
    self.learn_step_counter=0
    self.action_space=[i for i in range(self.n_actions)]

    self.memory=replay_buffer(mem_size,input_dims)

    self.q_eval=dueling_double_DQN(self.lr,self.n_actions,input_dims=self.input_dims,name='DDQN_qval',
                                    chkpt_dir=self.chkpt_dir)
    self.q_next=dueling_double_DQN(self.lr,self.n_actions,input_dims=self.input_dims,name='DDQN_qnext',
                          chkpt_dir=self.chkpt_dir)
  def choose_action(self,observation):
    if np.random.random()<self.epsilon:
      state=torch.tensor([observation],dtype=torch.float)
      _,advantage=self.q_eval.forward(state)

      # Pick the top 50 with highest advantage and place it into next action
      index=torch.topk(advantage,K)[1]

      #Rebuild an action space for the environment (n of K nonzero "one-hot encoder")
      # action array
      action=np.zeros(self.n_actions)
      action[index]=1
    else:
      index=np.random.choice(n,K,replace=False)
      # action array
      action=np.zeros(self.n_actions)
      action[index]=1
    return action

  def store_transition(self,state,action,reward,state_,done):
    self.memory.store_transition(state,action,reward,state_,done)

  def replace_target_network(self):
    if self.learn_step_counter%self.replace_target_cnt==0:
      self.q_next.load_state_dict(self.q_eval.state_dict())
  def decrement_epsilon(self):
    self.epsilon=self.epsilon-self.eps_dec if self.epsilon>self.eps_min else self.eps_min
  def save_models(self):
    self.q_eval.save_check_point()
    self.q_next.save_check_point()
  def load_models(self):
    self.q_eval.load_check_point()
    self.q_next.load_check_point()
  def learn(self):
    if self.memory.memory_counter<self.batch_size:
      return
    self.q_eval.optimizer.zero_grad()

    self.replace_target_network()

    state,action,reward,new_state,done = self.memory.sample_buffer(self.batch_size)

    states=torch.tensor(state).to(self.q_eval.device)
    actions=torch.tensor(action).to(self.q_eval.device)
    dones=torch.tensor(done).to(self.q_eval.device)
    rewards=torch.tensor(reward).to(self.q_eval.device)

    states_=torch.tensor(new_state).to(self.q_eval.device)

    indices=torch.arange(self.batch_size).unsqueeze(1)

    action_indices = torch.tensor([np.where(row == 1)[0].tolist() for row in actions])
    V_s,A_s=self.q_eval.forward(states)
    V_s_,A_s_=self.q_next.forward(states_)
    V_s_eval,A_s_eval=self.q_eval.forward(states_)

    q_pred=torch.add(V_s,(A_s-A_s.mean(dim=1,keepdim=True)))[indices,action_indices]
    q_next=torch.add(V_s_,(A_s_-A_s_.mean(dim=1,keepdim=True)))
    q_eval=torch.add(V_s_eval,(A_s_eval-A_s_eval.mean(dim=1,keepdim=True)))
    max_actions=torch.topk(q_eval,K,dim=1)[1]
    q_next[done]=0.0

    # q_target=rewards+self.gamma*q_next[indices,max_actions]
    Q_pred=torch.mean(q_pred,1)
    # Q_eval=torch.mean(q_eval,1)
    Q_target=rewards+self.gamma*torch.mean(q_next[indices,max_actions],1)

    loss=self.q_eval.loss(Q_target,Q_pred).to(self.q_eval.device)

    loss.backward()
    self.q_eval.optimizer.step()
    self.learn_step_counter+=1
    self.decrement_epsilon()



#-------------------------------------------


# **Testing some stuff**

# Testing gym with DDQN


In [28]:
n=len(mu)
K=50
episode=1

In [ ]:
import time
Best_MVO=[]
time_ls=[]
for i in range(len(date_ls)):
  scores,eps_history=[],[]
  avg_score_ls=[]
  MVO_ls=[]
  asset_ls=[]
  Max_MVO_ls=[]
  Last_MVO_ls=[]
  start_time=time.time()
  mu = np.loadtxt('{}/{}/mu'.format(path_to_data, date_ls[i]))
  Q = np.loadtxt('{}/{}/Q'.format(path_to_data, date_ls[i]))
  env=TrainEnv2(n,K,episode)
  load_check_point=False
  num_games=1000
  agent=Agent(gamma=0.8,epsilon=1,lr=0.001,input_dims=[1],n_actions=n,
              mem_size=100,eps_min=0.01,batch_size=64,eps_dec=1e-3,replace=round(num_games/3))
  if load_check_point:
    agent.load_models()
  for j in range(num_games):
    done=False
    observation =env.reset()
    score=0

    while not done:
      action=agent.choose_action(observation)
      observation_,reward,done,info=env.step(action)
      # Not sure if it works, I want to compare the historical data if it is the highest in the history
      MVO_ls.append(observation_)
      if len(MVO_ls)!=0:
        if observation>np.max(MVO_ls):
          reward=reward*2
      asset_index=np.nonzero(action)
      score+=reward
      agent.store_transition(observation,action,reward,observation_,int(done))

      agent.learn()
      observation=observation_
      # print("Observation",observation,"Action",action,"reward",reward)
    Max_MVO_ls.append(max(MVO_ls))
    print(max(MVO_ls))
    scores.append(score)
    avg_score=np.mean(scores[-100:])
    avg_score_ls.append(avg_score)
    Last_MVO_ls.append(observation)
    asset_ls.append(asset_index)
    print('episode',j,'score %.1f' %score,
          'average_score %.1f' %avg_score,
          'epsilon %.2f' %agent.epsilon,
          'MVO Value %6f' %observation,
          'Final Action',asset_index)
    if i>round(n/3) and i%round(n/3)==0:
      agent.save_models()
    eps_history.append(agent.epsilon)
  run_time=time.time()-start_time
  print("Run time is % 5f"% run_time)

  Best_MVO.append(max(MVO_ls))
  time_ls.append(run_time)
  # x=[i+1 for i in range(num_games)]



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Streaming output truncated to the last 5000 lines.
episode 909 score -1.0 average_score -0.0 epsilon 0.15 MVO Value -19.722479 Final Action (array([  5,  40,  42,  64,  83, 104, 124, 142, 144, 151, 159, 179, 204,
       285, 292, 312, 332, 335, 345, 385, 396, 405, 415, 424, 430, 499,
       507, 510, 527, 607, 613, 629, 711, 714, 718, 740, 743, 757, 785,
       812, 816, 819, 846, 855, 904, 929, 951, 955, 959, 966]),)
-19.491893082990753
episode 910 score -1.0 average_score -0.0 epsilon 0.15 MVO Value -19.789063 Final Action (array([  0,   9,  39,  43,  84,  91, 114, 117, 186, 236, 253, 262, 285,
       290, 291, 294, 307, 315, 342, 356, 383, 410, 455, 487, 505, 527,
       532, 534, 556, 567, 610, 613, 669, 716, 717, 727, 730, 747, 771,
       785, 790, 796, 811, 823, 842, 844, 922, 943, 956, 971]),)
-19.491893082990753
episode 911 score 1.0 average_score -0.0 epsilon 0.15 MVO Value -19.772975 Final Action (array([ 15,  79,  92,  99, 123, 127, 128, 130, 133, 217, 267, 272, 275,
      

In [27]:
Best_MVO,time_ls

NameError: ignored

In [ ]:
import matplotlib.pyplot as plt
plt.plot(Last_MVO_ls)
plt.xlabel("Num of the game")
plt.ylabel("Last MVO value at of the end of each game")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(Max_MVO_ls)
plt.xlabel("Num of the game")
plt.ylabel("Max MVO value through the num of game")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(avg_score_ls)
plt.xlabel("Num of game")
plt.ylabel("Reward Score at the end of episode")

In [ ]:

dqn = DQN()

print('\nCollecting experience...')
for i_episode in range(400):
    s = env.reset()
    ep_r = 0
    while True:
        a = dqn.choose_action(s)
        s_, r, done, info= env.step(a)
        print(a,s_,r,done,ep_r)
        # modify the reward
        # x, x_dot, theta, theta_dot = s_
        # r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8
        # r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
        # r = r1 + r2

        dqn.store_transition(s, a, r, s_)

        ep_r += r
        if dqn.memory_counter > MEMORY_CAPACITY:
            dqn.learn()
            if done:
                print('Ep: ', i_episode,
                      '| Ep_r: ', round(ep_r, 2))

        if done:
            break
        s = s_